# 1. Overview

# 2. Future Predictions

## 2.1 Import Libraries

In [6]:
import pandas as pd
import joblib
from prophet import Prophet


## 2.2 Define Global Variables

In [19]:
# Define the dictionary with file names and suppliers
company_tickers = {
    'CZMWY': 'Carl Zeiss Meditec',
    'LRCX': 'Lam Research Corporation',
    'TOELY': 'Tokyo Electron Limited'
}

regressors = ['Close_Carl Zeiss Meditec', 'Close_Lam Research Corporation', 
              'Close_Tokyo Electron Limited', 'COVID_Period', 
              'Geopolitical_Tension', 'Trade_Sanctions', 
              'Tech_Regulation', 'New_Product_Launch']

output_model_path_asml_suppliers = 'models/prophet_asml_combination_with_suppliers.pkl'
input_data = 'data/weekly_data_combination.csv'
future_weeks = 2


## 2.3 Code

### 2.3.1 Loading the data from notebook 04_data_preparation

In [3]:
weekly_data_combination = pd.read_csv(input_data, sep=';')

# Ensure the 'Date' column is in datetime format
weekly_data_combination['Date'] = pd.to_datetime(weekly_data_combination['Date'])

# Set the 'Date' column as the index to make it a DatetimeIndex
weekly_data_combination.set_index('Date', inplace=True)

# Explicitly set the frequency of the Date index
weekly_data_combination = weekly_data_combination.asfreq('W')


weekly_data_combination

,Close_ASML,Close_Carl Zeiss Meditec,Close_Lam Research Corporation,Close_Tokyo Electron Limited,COVID_Period,Geopolitical_Tension,Trade_Sanctions,Tech_Regulation,New_Product_Launch,Israel_Gaza_Conflict,Economic_Stimulus
Date,,,,,,,,,,,
2013-01-13,56.546021,26.057863,1.605489,7.480000,0,0,0,0,0,0,0
2013-01-20,57.203339,27.226337,1.637540,7.460000,0,0,0,0,0,0,0
2013-01-27,62.106480,27.674398,1.713177,7.406667,0,0,0,0,0,0,0
2013-02-03,66.858604,28.728659,1.782833,7.140000,0,0,0,0,0,0,0
2013-02-10,67.498169,29.343649,1.819157,6.853333,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2024-10-13,832.038757,72.779999,82.059998,86.790001,0,0,0,0,0,0,0
2024-10-20,721.585144,67.094002,74.139999,81.540001,0,0,0,0,0,0,0
2024-10-27,710.051941,67.529999,73.000000,76.709999,0,0,0,0,0,0,0


### 2.3.2 Loading Suppliers Pickles

In [4]:
# Create an empty dictionary to store the loaded models
models_suppliers = {}

# Iterate over each supplier in company_tickers to load its model
for ticker, company in company_tickers.items():
    # Define the file path to the model pickle file for the current supplier
    model_path = f'models/prophet_asml_supplier_{company.replace(" ", "_")}.pkl'
    
    # Load the model and store it in the dictionary using the supplier's name
    models_suppliers[company] = joblib.load(model_path)
    print(f'Prophet model for {company} loaded from {model_path}')

C:\Users\Usuario\anaconda3\envs\forecastingv2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


Prophet model for Carl Zeiss Meditec loaded from models/prophet_asml_supplier_Carl_Zeiss_Meditec.pkl
Prophet model for Lam Research Corporation loaded from models/prophet_asml_supplier_Lam_Research_Corporation.pkl
Prophet model for Tokyo Electron Limited loaded from models/prophet_asml_supplier_Tokyo_Electron_Limited.pkl


### 2.3.3 Train the model with new data

The prophet model that was trained in notebook 05_modelling, we are going to train it again with all the data without separating between train and test. 

In [12]:
prophet_model = Prophet(
    changepoint_prior_scale= 0.07, 
    daily_seasonality= False, 
    holidays_prior_scale= 0.1, 
    seasonality_mode= 'additive', 
    seasonality_prior_scale= 1.5, 
    weekly_seasonality= True, 
    yearly_seasonality= False
)

Modify the column names as required by prophet

In [13]:
train_data = weekly_data_combination.reset_index(names="ds").rename(columns={'Close_ASML':'y'})

Add regressors

In [14]:
for regressor in regressors:
    prophet_model.add_regressor(regressor)

Fit the model

In [15]:
prophet_model.fit(train_data)

12:45:19 - cmdstanpy - INFO - Chain [1] start processing
12:45:20 - cmdstanpy - INFO - Chain [1] done processing


### 2.3.4 Building the future dataframe

In [23]:
future_dataframe = prophet_model.make_future_dataframe(periods=future_weeks, freq='w')[-future_weeks:]
future_dataframe

,ds
618,2024-11-17
619,2024-11-24
